# wps_decompose_flow_vectors

#### wps_decompose_flow_vectors is a process that runs the [decompose_flow_vectors](https://github.com/pacificclimate/climate-explorer-data-prep#decompose_flow_vectors-create-normalized-unit-vector-fields-from-a-vic-routing-file) module of PCIC Climate Explorer Data Preparation Tools. Here, the client will try to connect to a remote Thunderbird instance using the url parameter.

In [1]:
from birdy import WPSClient
from pkg_resources import resource_filename
import requests
import os
from datetime import datetime
from wps_tools.utils import copy_http_content
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "thunderbird":
    os.chdir('../')

url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thunderbird/wps'
thunderbird = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [2]:
# NBVAL_IGNORE_OUTPUT
thunderbird.decompose_flow_vectors?

Signature:
thunderbird.decompose_flow_vectors(
    netcdf,
    variable,
    dest_file=None,
    loglevel='INFO',
)
Docstring:
Process an indexed flow direction netCDF into a vectored netCDF suitable for ncWMS display

Parameters
----------
netcdf : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF file
variable : string
    netCDF variable describing flow direction
dest_file : string
    destination netCDF file
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    output netCDF file
File:      ~/Desktop/thunderbird/</home/sangwonl/Desktop/thunderbird/venv/lib/python3.6/site-packages/birdy/client/base.py-4>
Type:      method


#### We can use the docstring to ensure we provide the appropriate parameters.

In [3]:
flow_vectors_file = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/sample_flow_parameters.nc"
variable = "Flow_Direction"
dest_file = "output.nc"
output = thunderbird.decompose_flow_vectors(netcdf=flow_vectors_file, variable=variable, dest_file=dest_file)

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [4]:
input_data = [
    direction
    for subarray in Dataset(flow_vectors_file).variables["Flow_Direction"]
    for direction in subarray
    if direction != "masked"
]
with NamedTemporaryFile(suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
    output_data = Dataset(copy_http_content(output.get()[0], tmp_file))
    output_eastward =  [
        x_magnitude 
        for subarray in output_data.variables["eastward_Flow_Direction"] 
        for x_magnitude in subarray 
        if x_magnitude != "masked"
    ]
    output_northward = [
        y_magnitude 
        for subarray in output_data.variables["northward_Flow_Direction"] 
        for y_magnitude in subarray 
        if y_magnitude != "masked"
    ]

In [5]:
# Check if input and output data sizes are matching

assert len(input_data) == len(output_eastward)
assert len(output_eastward) == len(output_northward)

In [6]:
# Check if input and output outlet positions are matching

outlets = [i for i in range(len(input_data)) if input_data[i] == 9]

# Outlets should have a flow direction of 0
eastward_outlets = [output_eastward[i] for i in range(len(input_data)) if i in outlets]
northward_outlets = [output_northward[i] for i in range(len(input_data)) if i in outlets]
expected_outlets = [0.0 for i in range(len(outlets))]

assert eastward_outlets == expected_outlets
assert northward_outlets == expected_outlets